# Install Required Software

In [14]:
#Mambaforge
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Mambaforge-$(uname)-$(uname -m).sh
!bash Mambaforge-$(uname)-$(uname -m).sh -b -u -p $HOME/mambaforge

#Trimmomatic, Fastqc, bowtie, samtools, rsem
!$HOME/mambaforge/bin/mamba install -y -c conda-forge -c bioconda trimmomatic fastqc bowtie samtools rsem multiqc

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 90.4M  100 90.4M    0     0  94.5M      0 --:--:-- --:--:-- --:--:--  210M
PREFIX=/home/jupyter/mambaforge
Unpacking payload ...
Extracting python-3.10.6-ha86cf86_0_cpython.tar.bz2
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2022.9.24-ha878542_0.tar.bz2
Extracting ld_impl_linux-64-2.39-hc81fddc_0.tar.bz2
Extracting libstdcxx-ng-12.2.0-h46fd767_19.tar.bz2
Extracting pybind11-abi-4-hd8ed1ab_3.tar.bz2
Extracting tzdata-2022f-h191b570_0.tar.bz2
Extracting libgomp-12.2.0-h65d4601_19.tar.bz2
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-ng-12.2.0-h65d4601_19.tar.bz2
Extracting bzip2-1.0.8-h7f98852_4.tar.bz2
Extracting c-ares-1.18.1

# Download Files

We need to create the folders we will use, as well as download the fastq and reference files that will be used for analysis. 

In [15]:
## Create Directories to hold files
!mkdir -p data
!mkdir -p data/raw_fastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc
!mkdir -p data/multiqc
!mkdir -p data/reference
!mkdir -p data/rsem

## Download Fastq files

# Download all runs from the Google folder bucket
!gsutil cp gs://hon_350/example_truncated_fastq/* data/raw_fastq

## Adapter sequences file for trimmomatic
!gsutil cp gs://hon_350/reference/TruSeq3-PE.fa data/reference/TruSeq3-PE.fa

## (Optional) Download reference files for building transcriptome index
#!gsutil cp gs://hon_350/referebce/GRCz11.dna_sm.toplevel.fa.gz data/reference/GRCz11.dna_sm.toplevel.fa.gz
#!gsutil cp gs://hon_350/referebce/GRCz11.108.gtf.gz data/reference/GRCz11.108.gtf.gz
#!echo 'unzipping...'
#!gzip -d data/reference/GRCz11.dna_sm.toplevel.fa.gz
#!gzip -d data/reference/GRCz11.107.gtf.gz
#!echo 'finished unzipping.'

## Download and unzip premade index files
!gsutil cp gs://hon_350/reference/rsem_grcz11.108_transcriptome.tar.gz data/reference/rsem_grcz11.108_transcriptome.tar.gz
!echo 'unzipping...'
!tar -xzvf data/reference/rsem_grcz11.108_transcriptome.tar.gz
!rm data/reference/rsem_grcz11.108_transcriptome.tar.gz
!echo 'finished unzipping.'

Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60184_S1_L004_R1_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60184_S1_L004_R2_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60185_S2_L004_R1_001.fastq.gz...
/ [3 files][  5.4 MiB/  5.4 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60185_S2_L004_R2_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60186_S3_L004_R1_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60186_S3_L004_R2_001.fastq.gz...
Copying gs://hon_350/example_truncated_fastq/trunc_SE6151_SA60187_S4_L004_R1_001.fastq.gz...
Copying gs://hon

# Trimmomatic

Trimmomatic is used to trim fastq files to remove adapters, and low quality score bases in reads.

A nice guide on trimmomatic parameters: https://datacarpentry.org/wrangling-genomics/03-trimming/index.html 

Trimmomatic works on one paired or single end run at a time.

However we have dozens of paired-end reads. 

To speed things up, this example 'pipes in' a list of run ids into trimmomatic to run sequentially.

In [29]:
# Run trimmomatic
# trimmomatic works on one paired-end reads file
# at a time. an example might look like this:

#!trimmomatic PE -threads 16 data/raw_fastq/trunc_SE6151_SA60191_S8_L004_R1_001.fastq.gz data/raw_fastq/trunc_SE6151_SA60191_S8_L004_R2_001.fastq.gz data/trimmed/SA60191_1_trimmed.fastq.gz data/trimmed/SA60191_1_orphans.fastq.gz data/trimmed/SA60191_2_trimmed.fastq.gz data/trimmed/SA60191_2_orphans.fastq.gz ILLUMINACLIP:data/reference/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

# for the sake of this example a list is 
# created using filenames in the fastq
# directory, and those names are piped in
# iteratively to the command function

# a list of the runs is made
!ls data/raw_fastq/*R1* |  sed 's|data/raw_fastq/||g' | sed 's|1_001.fastq.gz||g' > run_ids.txt

#what the list looks like
!cat run_ids.txt
!echo ' '

#the list is piped into trimmomatic
!cat run_ids.txt |  xargs -I {} trimmomatic PE -threads 16 data/raw_fastq/{}1_001.fastq.gz data/raw_fastq/{}2_001.fastq.gz data/trimmed/{}1_trimmed.fastq.gz data/trimmed/{}1_orphans.fastq.gz data/trimmed/{}2_trimmed.fastq.gz data/trimmed/{}2_orphans.fastq.gz ILLUMINACLIP:data/reference/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

#may not be necessary,
#but just to save space, delete raw fastq files after
!rm -r data/raw_fastq

trunc_SE6151_SA60184_S1_L004_R
trunc_SE6151_SA60185_S2_L004_R
trunc_SE6151_SA60186_S3_L004_R
trunc_SE6151_SA60187_S4_L004_R
trunc_SE6151_SA60188_S5_L004_R
trunc_SE6151_SA60189_S6_L004_R
trunc_SE6151_SA60190_S7_L004_R
trunc_SE6151_SA60191_S8_L004_R
 
TrimmomaticPE: Started with arguments:
 -threads 2 data/raw_fastq/trunc_SE6151_SA60184_S1_L004_R1_001.fastq.gz data/raw_fastq/trunc_SE6151_SA60184_S1_L004_R2_001.fastq.gz data/trimmed/trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz data/trimmed/trunc_SE6151_SA60184_S1_L004_R1_orphans.fastq.gz data/trimmed/trunc_SE6151_SA60184_S1_L004_R2_trimmed.fastq.gz data/trimmed/trunc_SE6151_SA60184_S1_L004_R2_orphans.fastq.gz ILLUMINACLIP:data/reference/TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detec

# Fastqc

Fastqc is a tool to inspect the quality of reads in fastq files.

In [30]:
# Run fastqc
!fastqc -o data/fastqc data/trimmed/*_trimmed.fastq.gz

# Example of in-window display of fastqc output file (SA60184).
from IPython.display import IFrame
IFrame(src='./data/fastqc/trunc_SE6151_SA60184_S1_L004_R1_trimmed_fastqc.html', width=800, height=600)


Started analysis of trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 5% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 10% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 15% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 20% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 25% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 30% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 35% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 40% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 45% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 50% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 55% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 60% complete for trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz
Approx 65% complete for trunc_SE6151_SA60184_S1_L004_R1_

In [31]:
!multiqc -f data/fastqc -o data/multiqc

import pandas as pd
dframe = pd.read_csv("data/multiqc/multiqc_data/multiqc_fastqc.txt", sep='\t')
display(dframe)


  /// ]8;id=336355;https://multiqc.info\MultiQC]8;;\ 🔍 | v1.13

|           multiqc | Search path : /home/jupyter/Example_Whim_Notebook/data/fastqc
|         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 32/32  0m  
|            fastqc | Found 16 reports
|           multiqc | Compressing plot data
|           multiqc | Deleting    : data/multiqc/multiqc_report.html   (-f was specified)
|           multiqc | Deleting    : data/multiqc/multiqc_data   (-f was specified)
|           multiqc | Report      : data/multiqc/multiqc_report.html
|           multiqc | Data        : data/multiqc/multiqc_data
|           multiqc | MultiQC complete


,Sample,Filename,File type,Encoding,Total Sequences,Sequences flagged as poor quality,Sequence length,%GC,total_deduplicated_percentage,avg_sequence_length,...,per_base_sequence_quality,per_tile_sequence_quality,per_sequence_quality_scores,per_base_sequence_content,per_sequence_gc_content,per_base_n_content,sequence_length_distribution,sequence_duplication_levels,overrepresented_sequences,adapter_content
0,trunc_SE6151_SA60184_S1_L004_R1,trunc_SE6151_SA60184_S1_L004_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,21329.0,0.0,150-151,38.0,78.194008,150.992545,...,pass,warn,pass,fail,warn,pass,warn,pass,warn,pass
1,trunc_SE6151_SA60184_S1_L004_R2,trunc_SE6151_SA60184_S1_L004_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,21329.0,0.0,150-151,41.0,87.866285,150.987201,...,pass,warn,pass,warn,pass,pass,warn,pass,warn,pass
2,trunc_SE6151_SA60185_S2_L004_R1,trunc_SE6151_SA60185_S2_L004_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,19635.0,0.0,150-151,37.0,85.663356,150.991851,...,pass,warn,pass,fail,warn,pass,warn,pass,warn,pass
3,trunc_SE6151_SA60185_S2_L004_R2,trunc_SE6151_SA60185_S2_L004_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,19635.0,0.0,150-151,39.0,93.027757,150.986096,...,pass,warn,pass,warn,warn,pass,warn,pass,pass,pass
4,trunc_SE6151_SA60186_S3_L004_R1,trunc_SE6151_SA60186_S3_L004_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,17563.0,0.0,150-151,40.0,81.500883,150.993224,...,pass,warn,pass,fail,fail,pass,warn,pass,warn,pass
5,trunc_SE6151_SA60186_S3_L004_R2,trunc_SE6151_SA60186_S3_L004_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,17563.0,0.0,150-151,42.0,90.229460,150.987474,...,pass,fail,pass,warn,warn,pass,warn,pass,warn,pass
6,trunc_SE6151_SA60187_S4_L004_R1,trunc_SE6151_SA60187_S4_L004_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,21689.0,0.0,150-151,35.0,77.762921,150.991885,...,pass,warn,pass,fail,warn,pass,warn,pass,warn,pass
7,trunc_SE6151_SA60187_S4_L004_R2,trunc_SE6151_SA60187_S4_L004_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,21689.0,0.0,150-151,39.0,88.288994,150.986030,...,pass,warn,pass,warn,warn,pass,warn,pass,warn,pass
8,trunc_SE6151_SA60188_S5_L004_R1,trunc_SE6151_SA60188_S5_L004_R1_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,24736.0,0.0,150-151,80.0,3.553525,150.990944,...,pass,warn,pass,fail,fail,pass,warn,fail,fail,pass
9,trunc_SE6151_SA60188_S5_L004_R2,trunc_SE6151_SA60188_S5_L004_R2_trimmed.fastq.gz,Conventional base calls,Sanger / Illumina 1.9,24736.0,0.0,150-151,76.0,20.015362,150.986376,...,pass,fail,pass,fail,fail,pass,warn,fail,fail,pass


# Rsem

In order to map reads to genes, and quantify read counts, an index must be created. In this case, we are using rsem for index creation and quantifying read counts.

The output and error log for this software is too large to display in the cell below, so is stored in a text log file 'rsem/rsem_log.txt' instead. 

For the example truncated set this can finish within a few minutes, but with the actual data set this will take around eight hours to fully run.

In [6]:
## Build transcriptome reference

#!rsem-prepare-reference --gtf data/reference/GRCz11.108.gtf --bowtie -p 8 data/reference/GRCz11.dna_sm.toplevel.fa data/reference/zebrafish_rsem/GRCz11_108 

In [7]:
# to make writing commands easier,
# again we will make a list and 'pipe' the list
# into rsem.
# first make a list
!ls data/trimmed/*R1*trimmed* |  sed 's|data/trimmed/||g' | sed 's|.fastq.gz||g' > trimmed_ids.txt
# then run rsem
!echo 'running calculate counts...'
!cat trimmed_ids.txt |  xargs -I {} rsem-calculate-expression -p 16 --forward-prob 0 data/trimmed/{}.fastq.gz data/reference/zebrafish_rsem/GRCz11_108 data/rsem/{} >data/rsem/rsem_log.txt 2>&1
!echo 'finished'

running calculate counts...
finished


# Read count file outputs

Rsem outputs readcount files (XXXXX.genes.results) which are tables that contain various information, including readcounts in transcripts per kilobase million (TPM) and fragments per kilobase of exon per million mapped fragments (FPKM) format.

In [8]:
# Example of read count table output
# (sorted by top 10 expressed genes)
!head -1 $(ls data/rsem/*.genes.results | head -n1) | column -t
!sort -nrk 6 $(ls data/rsem/*.genes.results | head -n1) |  column -t | head -10  | column -t

gene_id  transcript_id(s)  length  effective_length  expected_count  TPM  FPKM
ENSDARG00000114494  ENSDART00000187218  165.00   46.58    102081.87  128401.41  170718.92
ENSDARG00000087732  ENSDART00000122203  298.00   179.58   358926.10  117105.31  155699.95
ENSDARG00000110859  ENSDART00000193467  165.00   46.58    26641.94   33510.97   44555.24
ENSDARG00000109433  ENSDART00000180315  165.00   46.58    26641.94   33510.97   44555.24
ENSDARG00000083063  ENSDART00000118247  165.00   46.58    26641.94   33510.97   44555.24
ENSDARG00000081270  ENSDART00000117461  322.00   203.58   87739.30   25251.60   33573.82
ENSDARG00000089382  ENSDART00000126982  1212.00  1093.58  467080.56  25024.96   33272.48
ENSDARG00000082008  ENSDART00000116893  96.00    6.95     2714.12    22876.81   30416.37
ENSDARG00000081580  ENSDART00000116507  72.00    1.95     723.00     21707.91   28862.23
ENSDARG00000082753  ENSDART00000115891  952.00   833.58   285374.88  20058.59   26669.34


# Upload files to bucket
In order for the files to be shared and downloaded, we need to upload them to the google bucket.

In [9]:
## Upload to bucket
!gsutil cp data/rsem/*.genes.results gs://hon_350/example_truncated_results/

Copying file://data/rsem/SE8159_SA161910_S1_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/SE8159_SA161911_S2_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/SE8159_SA161912_S3_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/SE8159_SA161913_S4_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
- [4 files][ 10.6 MiB/ 10.6 MiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://data/rsem/SE8159_SA161914_S5_L007_R1_trimmed.genes.results [Content-Type=application/octet-stream]...
Copying file://data/rsem/SE8159_SA161915_S6_L007_R1_trimmed.genes.results [Content